In [1]:
import pygame
import random
import cv2
import mediapipe as mp

# Initialize Pygame
pygame.init()

# Screen dimensions
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

# Game title and background
pygame.display.set_caption("Asteroid Game")
background_img = pygame.image.load('C:/Users/ADMIN/projects/Asteroid game/background.png')

# Load images
spaceship_img = pygame.image.load('C:/Users/ADMIN/projects/Asteroid game/spaceship.PNG')
asteroid_img = pygame.image.load('C:/Users/ADMIN/projects/Asteroid game/Asteroid_01.png')

# Scale images if necessary
spaceship_img = pygame.transform.scale(spaceship_img, (64, 64))  # Resize spaceship
asteroid_img = pygame.transform.scale(asteroid_img, (64, 64))    # Resize asteroid

# Spaceship initial position
spaceship_x = screen_width // 2 - 32
spaceship_y = screen_height - 100
spaceship_x_change = 0

# Asteroid initial position
asteroid_x = random.randint(0, screen_width - 64)
asteroid_y = random.randint(50, 150)
asteroid_y_change = 1  # Slowed down from 5 to 1

# Bullet settings
bullet_x = 0
bullet_y = spaceship_y
bullet_y_change = 10
bullet_state = "ready"  # "ready" means bullet is not visible, "fire" means bullet is fired

# Score
score_value = 0
font = pygame.font.Font('freesansbold.ttf', 32)

# Initialize MediaPipe pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# OpenCV video capture
cap = cv2.VideoCapture(0)

def show_score(x, y):
    score = font.render("Score: " + str(score_value), True, (255, 255, 255))
    screen.blit(score, (x, y))

def draw_spaceship(x, y):
    screen.blit(spaceship_img, (x, y))

def draw_asteroid(x, y):
    screen.blit(asteroid_img, (x, y))

def fire_bullet(x, y):
    global bullet_state
    bullet_state = "fire"
    pygame.draw.rect(screen, (255, 0, 0), (x + 28, y, 8, 20))  # A small cylindrical bullet

def is_collision(asteroid_x, asteroid_y, bullet_x, bullet_y):
    distance = ((asteroid_x - bullet_x) ** 2 + (asteroid_y - bullet_y) ** 2) ** 0.5
    return distance < 27

def game_over_text():
    over_font = pygame.font.Font('freesansbold.ttf', 64)
    game_over = over_font.render("GAME OVER", True, (255, 0, 0))
    screen.blit(game_over, (screen_width // 4, screen_height // 3))

def detect_movements(frame):
    global spaceship_x_change, bullet_state, bullet_x

    # Process the frame using MediaPipe Pose
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        # Extract landmarks for nose and ears
        landmarks = result.pose_landmarks.landmark

        # Get positions of important landmarks
        nose_y = landmarks[mp_pose.PoseLandmark.NOSE].y * screen_height
        right_ear_y = landmarks[mp_pose.PoseLandmark.LEFT_EAR].y * screen_height
        left_ear_y = landmarks[mp_pose.PoseLandmark.RIGHT_EAR].y * screen_height

        # Detect head tilt based on the vertical difference between ears
        ear_tilt_diff = left_ear_y - right_ear_y

        # Apply threshold to detect significant head tilt
        tilt_threshold = 10  # Adjust this for sensitivity

        if ear_tilt_diff > tilt_threshold:  # Head tilted to the left
            spaceship_x_change = -5  # Move left
        elif ear_tilt_diff < -tilt_threshold:  # Head tilted to the right
            spaceship_x_change = 5  # Move right
        else:
            spaceship_x_change = 0  # No significant tilt

        # Control bullet firing using nose position (simple head tilt can fire bullets)
        if abs(ear_tilt_diff) < tilt_threshold and bullet_state == "ready":
            bullet_x = spaceship_x
            fire_bullet(bullet_x, bullet_y)

# Game loop
running = True
while running:
    # Fill the background with the custom image
    screen.blit(background_img, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update spaceship position
    spaceship_x += spaceship_x_change
    spaceship_x = max(0, min(spaceship_x, screen_width - 64))  # Keep spaceship within screen boundaries

    # Update asteroid position
    asteroid_y += asteroid_y_change
    if asteroid_y > screen_height:
        asteroid_x = random.randint(0, screen_width - 64)
        asteroid_y = random.randint(50, 150)

    # Bullet movement
    if bullet_state == "fire":
        fire_bullet(bullet_x, bullet_y)
        bullet_y -= bullet_y_change
        if bullet_y <= 0:
            bullet_y = spaceship_y
            bullet_state = "ready"

    # Bullet collision detection
    collision = is_collision(asteroid_x, asteroid_y, bullet_x, bullet_y)
    if collision:
        bullet_y = spaceship_y
        bullet_state = "ready"
        asteroid_x = random.randint(0, screen_width - 64)
        asteroid_y = random.randint(50, 150)
        score_value += 1

    # Draw the spaceship and asteroid
    draw_spaceship(spaceship_x, spaceship_y)
    draw_asteroid(asteroid_x, asteroid_y)
    
    # Show the score
    show_score(10, 10)

    # Game over condition
    if asteroid_y > spaceship_y and abs(asteroid_x - spaceship_x) < 50:
        game_over_text()
        pygame.display.update()
        pygame.time.wait(2000)
        running = False

    # OpenCV part - camera capture
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 1)  # Mirror image for better control
        detect_movements(frame)  # Detect movements (head tilt)

        # Show the camera feed window
        cv2.imshow('Camera Feed', frame)

    pygame.display.update()
    pygame.time.Clock().tick(60)

    # Close window on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\ADMIN\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
